# Azure AI Agents with Model Context Protocol (MCP) Support

이 노트북은 Model Context Protocol (MCP) 도구를 지원하는 Azure AI Agents를 사용하여 외부 MCP 서버를 활용해 향상된 기능을 제공하는 지능형 에이전트를 만드는 방법을 보여줍니다.


## 필수 NuGet 패키지 설치

먼저, Azure AI Agents와 작업하기 위한 핵심 기능을 제공하는 Azure AI Agents Persistent 패키지를 설치해야 합니다.


## 키리스 인증의 장점

이 노트북은 **키리스 인증**을 시연하며 다음과 같은 여러 장점을 제공합니다:
- ✅ **API 키 관리 불필요** - Azure 기반 인증 사용
- ✅ **보안 강화** - 코드나 설정에 비밀을 저장하지 않음
- ✅ **자동 자격 증명 갱신** - Azure가 자격 증명 수명 주기를 관리
- ✅ **역할 기반 접근** - 세분화된 권한을 위한 Azure RBAC 사용

`DefaultAzureCredential`은 자동으로 가장 적합한 자격 증명 소스를 사용합니다:
1. 관리형 ID (Azure에서 실행 중일 때)
2. Azure CLI 자격 증명 (개발 중일 때)
3. Visual Studio 자격 증명
4. 환경 변수 (구성된 경우)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure 서비스와의 인증을 위해 DefaultAzureCredential을 사용하여 Azure Identity 패키지를 설치하세요.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## 필수 네임스페이스 가져오기

Azure AI Agents와 Azure Identity에 필요한 네임스페이스를 가져옵니다.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI 에이전트 클라이언트 설정 (키 없는 인증)

**키 없는 인증**을 사용하여 구성 변수를 설정하고 PersistentAgentsClient를 생성합니다:
- **projectEndpoint**: Azure AI Foundry 프로젝트 엔드포인트
- **modelDeploymentName**: 배포된 AI 모델의 이름 (GPT-4.1 nano)
- **mcpServerUrl**: MCP 서버의 URL (Microsoft Learn API)
- **mcpServerLabel**: MCP 서버를 식별하기 위한 라벨
- **DefaultAzureCredential**: 관리 ID, Azure CLI 또는 기타 자격 증명 소스를 사용 (API 키 불필요)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP 도구 정의 생성

Microsoft Learn MCP 서버에 연결하는 MCP 도구 정의를 생성합니다. 이를 통해 에이전트가 Microsoft Learn 콘텐츠와 문서에 접근할 수 있습니다.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI 에이전트 생성하기

지정된 모델과 MCP 도구를 사용하여 지속적인 AI 에이전트를 생성합니다. 에이전트는 다음과 같이 구성됩니다:
- GPT-4.1 나노 모델
- MCP 도구를 활용하도록 지시
- Microsoft Learn MCP 서버에 대한 접근 권한


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## 스레드 생성 및 메시지 보내기

대화 스레드를 생성하고 Azure OpenAI와 OpenAI의 차이에 대해 묻는 사용자 메시지를 보내세요. 이는 에이전트가 MCP 도구를 사용하여 정확한 정보를 제공할 수 있는 능력을 테스트합니다.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP 도구 리소스 구성 (키리스)

MCP 도구 리소스를 설정합니다. 완전히 키리스 방식으로 설정하려면 MCP 서버가 Azure ID 기반 인증을 지원하는 경우 사용자 지정 헤더를 제거할 수 있습니다. 아래 예시는 필요할 경우 헤더를 추가하는 방법을 보여주지만, 키리스 시나리오에서는 이러한 헤더가 필요하지 않을 수 있습니다.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## 에이전트 실행 시작

사용자의 메시지를 처리하기 위해 실행을 생성하고 시작합니다. 에이전트는 설정된 MCP 도구와 리소스를 사용하여 응답을 생성합니다.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## 실행 모니터링 및 도구 승인 처리 (키리스)

에이전트 실행 상태를 모니터링하고 필요한 도구 승인을 처리합니다. 이 루프는 다음을 수행합니다:
1. 실행이 완료되거나 조치가 필요할 때까지 대기
2. MCP 도구 호출이 필요할 경우 자동으로 승인
3. 키리스 인증의 경우, MCP 서버가 Azure ID를 지원하면 헤더가 필요하지 않을 수 있음


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## 대화 결과 표시

스레드의 모든 메시지를 가져와 사용자 질문과 상담원의 응답을 모두 표시합니다. 메시지는 시간 순서대로 타임스탬프와 역할 표시와 함께 표시됩니다.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
